In [1]:
#load libraries and the classification model

import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import shutil
from PIL import Image
from tensorflow.keras.models import load_model
model = load_model('avo_model.keras')
from ultralytics import YOLO


C:\Users\johnl\AppData\Local\Temp\ipykernel_2376\1336455791.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


First we run the yolo model to identify the kind of avo. Since we can only predict Hass, maybe I need something to account for this. 

Next we move these images into a separate checking folder. This is to make the rest of the code sustainable and work across all devices consistently. This will move all the Hass crops into a separate checking folder (or shepards at a later date, may need an if else or something) and then delete the runs folder to avoid having the future cops go into 'predict2' and then having to keep track of the number of predicts.

In [12]:
model_yolo = YOLO("avoID.pt")

model_yolo.predict(
        source='Pictures/1034A',
        save_crop=True,
        save=True,
        conf=0.5
    )
#!yolo predict model=avoID.pt source='Pictures/1020' conf = 0.5 save_crop=True save
shutil.move('runs/detect/predict/crops/Hass','checking_folder/') #need to add a line if there are no hass' detected
shutil.rmtree('runs/')


image 1/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0022.jpg: 640x512 1 Hass, 50.2ms
image 2/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0023.jpg: 640x512 1 Hass, 9.6ms
image 3/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0024.jpg: 640x512 1 Hass, 9.4ms
image 4/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0025.jpg: 640x512 1 Hass, 9.9ms
image 5/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0026.jpg: 640x512 1 Hass, 9.7ms
image 6/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0027.jpg: 640x512 1 Hass, 9.6ms
image 7/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0028.jpg: 640x512 1 Hass, 9.7ms
image 8/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\IMG-20250517-WA0029.jpg: 640x512 1 Hass, 10.0ms
image 9/9 c:\Users\johnl\Documents\GitHub\avoChecker\Pictures\1034A\WhatsApp 

The next steps: 
1) Define a function to resize the images inputed
2) open checking_folder and store the names of each file in a list
3) loop through that list and resize each image
4) run the image through the algo and save the score
5) get an average of the final score as the score for the avos

In [13]:
##Lite model testing

##Ok this works well and is a little faster

def prepare_input(image_path, target_size=(256, 256)):
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

model_lite = tf.lite.Interpreter(model_path="avo_model.tflite")
model_lite.allocate_tensors()
input_details = model_lite.get_input_details()
output_details = model_lite.get_output_details()

def predict_tflite(input_data):
    model_lite.set_tensor(input_details[0]['index'], input_data)
    model_lite.invoke()
    return model_lite.get_tensor(output_details[0]['index'])

avo_pics = os.listdir('checking_folder/')

avo_score = [] #empty list to store the avo score

#Loop through all the pics in the folder to get the individual predictions for each avo
for pic in avo_pics:
    avo = prepare_input('checking_folder/' + pic)
    avo_score.append(predict_tflite(avo))

shutil.rmtree('checking_folder/') #delete the checking_folder to ensure we can use exactly the same code in the futur

avo_score = np.concatenate(avo_score, axis=0 )
avo_score.mean()/2

2.4809017181396484